In [1]:
# Path setup so Python can find the compiled _earcore module and the Python package
import sys, os
from pathlib import Path

def add_paths():
    cwd = Path.cwd()
    # Walk up to locate eartrainer/eartrainer_Cpp/{build,python}
    for base in [cwd, *cwd.parents]:
        build = base / 'eartrainer' / 'eartrainer_Cpp' / 'build'
        py = base / 'eartrainer' / 'eartrainer_Cpp' / 'python'
        added = False
        if build.exists():
            sys.path.insert(0, str(build))
            added = True
        if py.exists():
            sys.path.insert(0, str(py))
            added = True
        if added:
            return build, py
    return None, None

build_path, py_path = add_paths()
print('Using build path:', build_path)
print('Using python path:', py_path)


Using build path: /Users/itamarshamir/Projects/ear_trainer/eartrainer/eartrainer_Cpp/build
Using python path: /Users/itamarshamir/Projects/ear_trainer/eartrainer/eartrainer_Cpp/python


In [2]:
import sys
import pathlib

PROJECT_ROOT = pathlib.Path().resolve()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from eartrainer.session_engine import SessionEngine
from eartrainer import models

print("SessionEngine ready")

SessionEngine ready


In [3]:
def run_adaptive_bout(track_levels=None, n_questions=6, seed=7):
    engine = SessionEngine()
    levels = list(track_levels or [])
    spec = models.SessionSpec(
        version="v1",
        drill_kind="adaptive",
        key="",
        #key="C major",
        #range=[48, 72],
        #tempo_bpm=96,
        n_questions=n_questions,
        generation="adaptive",
        #assistance_policy={"Replay": 0},
        sampler_params={"track_levels": levels},
        seed=seed,
        adaptive=True,
        track_levels=levels,
    )
    session_id = engine.create_session(spec)
    summary = None
    timeline = []
    key = engine.session_key(session_id)
    print("Session key:", key)
    prompt = engine.orientation_prompt(session_id)
    print("Orientation prompt:", prompt)


    while True:
        next_item = engine.next_question(session_id)
        if isinstance(next_item, models.SessionSummary):
            summary = next_item
            break

        bundle = next_item
        print(bundle)
        metrics = models.ResultMetrics(rt_ms=1200, attempts=1, question_count=1)
        report = models.ResultReport(
            question_id=bundle.question_id,
            final_answer=bundle.correct_answer,
            correct=True,
            metrics=metrics,
        )
        timeline.append(bundle.question_id)

        next_payload = engine.submit_result(session_id, report)
        if isinstance(next_payload, models.SessionSummary):
            summary = next_payload
            break

    memory = engine.end_session(session_id)
    diagnostics = engine.adaptive_diagnostics(session_id)

    return {
        "session_id": session_id,
        "timeline": timeline,
        "summary": summary,
        "memory": memory,
        "diagnostics": diagnostics,
    }

In [4]:
import sys, eartrainer
print("exe:", sys.executable)
print("eartrainer location:", eartrainer.__file__)
from eartrainer import SessionEngine
print(hasattr(SessionEngine(), "adaptive_diagnostics"))


exe: /Users/itamarshamir/miniconda3/envs/eartrainer_cpp/bin/python
eartrainer location: /Users/itamarshamir/Projects/ear_trainer/eartrainer/eartrainer_Cpp/python/eartrainer/__init__.py
True


In [9]:
from pprint import pprint

result = run_adaptive_bout(track_levels=[11,111], n_questions=5, seed=42)

print("Session ID:", result["session_id"])
print("Timeline:", result["timeline"])
print("\nSummary totals:")
pprint(result["summary"].totals)

memory = result["memory"]
if memory.adaptive:
    print("\nAdaptive Memory:")
    pprint(
        {
            "bout_average": memory.adaptive.bout_average,
            "level_up": memory.adaptive.level_up,
            "graduate_threshold": memory.adaptive.graduate_threshold,
            "level": memory.adaptive.level.__dict__ if memory.adaptive.level else None,
        }
    )
    print("\nDrill EMAs:")
    for drill_id, drill_info in memory.adaptive.drills.items():
        pprint({drill_id: {"family": drill_info.family, "ema_score": drill_info.ema_score}})
else:
    print("No adaptive data returned.")

print(f"="*60)
print(f"=== ADAPTIVE DIAGNOSTICS")
print(f"="*60)
pprint(result["diagnostics"])

Session key: Bb major
Orientation prompt: Prompt(modality='midi-clip', midi_clip=MidiClip(ppq=480, tempo_bpm=96, length_ticks=2859, tracks=[MidiTrack(name='prompt', channel=0, program=0, events=[MidiEvent(t=0, type='note_on', note=58, vel=90, control=None, value=None), MidiEvent(t=246, type='note_off', note=58, vel=None, control=None, value=None), MidiEvent(t=246, type='note_on', note=60, vel=90, control=None, value=None), MidiEvent(t=492, type='note_off', note=60, vel=None, control=None, value=None), MidiEvent(t=492, type='note_on', note=62, vel=90, control=None, value=None), MidiEvent(t=738, type='note_off', note=62, vel=None, control=None, value=None), MidiEvent(t=738, type='note_on', note=63, vel=90, control=None, value=None), MidiEvent(t=984, type='note_off', note=63, vel=None, control=None, value=None), MidiEvent(t=984, type='note_on', note=65, vel=90, control=None, value=None), MidiEvent(t=1230, type='note_off', note=65, vel=None, control=None, value=None), MidiEvent(t=1230, typ

In [6]:
result['summary'].totals

{'adaptive_bout_score': 0.9924999999999999,
 'adaptive_drill_score_map': {'MELODY60': {'ema_score': None,
   'family': 'melody'},
  'MELODY60_STEP1': {'ema_score': 0.9925, 'family': 'melody'},
  'MELODY60_STEP2': {'ema_score': 0.9924999999999999, 'family': 'melody'}},
 'adaptive_drill_scores': [0.9925, 0.9924999999999999, None],
 'adaptive_level_current': 111,
 'adaptive_level_suggested': 112,
 'adaptive_level_track': 'melody_levels',
 'adaptive_level_up': True,
 'adaptive_level_up_threshold': 0.8,
 'avg_rt_ms': 1200,
 'correct': 5,
 'incorrect': 0}

In [7]:
result

{'session_id': 'sess-1',
 'timeline': ['q-001', 'q-002', 'q-003', 'q-004', 'q-005'],
 'summary': SessionSummary(session_id='sess-1', totals={'adaptive_bout_score': 0.9924999999999999, 'adaptive_drill_score_map': {'MELODY60': {'ema_score': None, 'family': 'melody'}, 'MELODY60_STEP1': {'ema_score': 0.9925, 'family': 'melody'}, 'MELODY60_STEP2': {'ema_score': 0.9924999999999999, 'family': 'melody'}}, 'adaptive_drill_scores': [0.9925, 0.9924999999999999, None], 'adaptive_level_current': 111, 'adaptive_level_suggested': 112, 'adaptive_level_track': 'melody_levels', 'adaptive_level_up': True, 'adaptive_level_up_threshold': 0.8, 'avg_rt_ms': 1200, 'correct': 5, 'incorrect': 0}, by_category=[{'label': 'adaptive', 'score': 1.0}], results=[{'correct': True, 'question_id': 'q-001', 'rt_ms': 1200, 'score': 1.0}, {'correct': True, 'question_id': 'q-002', 'rt_ms': 1200, 'score': 1.0}, {'correct': True, 'question_id': 'q-003', 'rt_ms': 1200, 'score': 1.0}, {'correct': True, 'question_id': 'q-004', 'r